## Text Query Embedding using MuQ-MuLan

MuQ-MuLan은 영어와 중국어 텍스트를 모두 지원합니다.

Template:
```
{genre}, {mood} mood, {tempo or energy}, 
{main instruments}, {vocal info + language}, 
suitable for {use-case or scene}
```


In [2]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("MPS available:", torch.backends.mps.is_available())

CUDA available: False
MPS available: True


In [4]:
from muq import MuQMuLan


# 디바이스 설정
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"사용 디바이스: {device}")

# MuQ-MuLan 모델 로드
model = MuQMuLan.from_pretrained("OpenMuQ/MuQ-MuLan-large")
model = model.to(device).eval()

print("MuQ-MuLan 모델 로드 완료!")


사용 디바이스: mps


/Users/gwagdaehun/dev/personal/ai-playground/music-rag/.venv/lib/python3.13/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


MuQ-MuLan 모델 로드 완료!


In [5]:
def get_text_embedding(query: str) -> torch.Tensor:
    """
    텍스트 쿼리를 MuQ-MuLan 임베딩으로 변환
    
    Args:
        query: 음악을 설명하는 텍스트 쿼리 (영어 또는 중국어)
        
    Returns:
        텍스트 임베딩 벡터
    """
    with torch.no_grad():
        # MuQ-MuLan은 texts 파라미터로 리스트를 받음
        text_emb = model(texts=[query])
    
    return text_emb


def get_text_embeddings_batch(queries: list) -> torch.Tensor:
    """
    여러 텍스트 쿼리를 한 번에 MuQ-MuLan 임베딩으로 변환
    
    Args:
        queries: 음악을 설명하는 텍스트 쿼리 리스트
        
    Returns:
        텍스트 임베딩 벡터들 (batch)
    """
    with torch.no_grad():
        text_emb = model(texts=queries)
    
    return text_emb


In [6]:
# 예시 쿼리 (템플릿 형식) - 영어
# {genre}, {mood} mood, {tempo or energy}, 
# {main instruments}, {vocal info + language}, 
# suitable for {use-case or scene}

example_queries_en = [
    # Electronic/Dance
    "Electronic, energetic mood, fast tempo, "
    "synthesizer and drum machine, no vocals, "
    "suitable for workout or gaming",
    
    # Lo-fi/Chill
    "Lo-fi hip hop, relaxed mood, slow tempo, "
    "piano and vinyl crackle, no vocals, "
    "suitable for studying or late night work",
    
    # Pop/Happy
    "Pop, happy mood, medium tempo, "
    "guitar and drums, female vocals in Korean, "
    "suitable for morning routine or commute",
    
    # Jazz/Romantic
    "Jazz, romantic mood, slow tempo, "
    "saxophone and piano, male vocals in English, "
    "suitable for dinner or date night",
    
    # Rock/Intense
    "Rock, intense mood, fast energy, "
    "electric guitar and heavy drums, male vocals in Korean, "
    "suitable for driving or exercise",
]

# 중국어 쿼리 예시 (MuQ-MuLan은 중국어도 지원)
example_queries_zh = [
    "古典音乐，平静的心情，慢节奏，钢琴和弦乐，无人声，适合阅读或冥想",  # Classical, calm, piano & strings
    "电子舞曲，充满活力，快节奏，合成器，无人声，适合运动或派对",  # EDM, energetic, synth
]

print(f"영어 쿼리: {len(example_queries_en)}개")
print(f"중국어 쿼리: {len(example_queries_zh)}개")


영어 쿼리: 5개
중국어 쿼리: 2개


In [7]:
# 영어 텍스트 쿼리 임베딩 생성
print("=== 영어 쿼리 임베딩 ===\n")
text_embeddings_en = {}

for i, query in enumerate(example_queries_en):
    emb = get_text_embedding(query)
    text_embeddings_en[f"query_en_{i}"] = emb
    print(f"✓ Query {i}: {query[:50]}...")
    print(f"  - shape: {emb.shape}")
    print()

print(f"총 {len(text_embeddings_en)}개 영어 쿼리 임베딩 완료!")


=== 영어 쿼리 임베딩 ===



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

✓ Query 0: Electronic, energetic mood, fast tempo, synthesize...
  - shape: torch.Size([1, 512])

✓ Query 1: Lo-fi hip hop, relaxed mood, slow tempo, piano and...
  - shape: torch.Size([1, 512])

✓ Query 2: Pop, happy mood, medium tempo, guitar and drums, f...
  - shape: torch.Size([1, 512])

✓ Query 3: Jazz, romantic mood, slow tempo, saxophone and pia...
  - shape: torch.Size([1, 512])

✓ Query 4: Rock, intense mood, fast energy, electric guitar a...
  - shape: torch.Size([1, 512])

총 5개 영어 쿼리 임베딩 완료!


In [8]:
# 중국어 텍스트 쿼리 임베딩 생성
print("=== 중국어 쿼리 임베딩 ===\n")
text_embeddings_zh = {}

for i, query in enumerate(example_queries_zh):
    emb = get_text_embedding(query)
    text_embeddings_zh[f"query_zh_{i}"] = emb
    print(f"✓ Query {i}: {query[:30]}...")
    print(f"  - shape: {emb.shape}")
    print()

print(f"총 {len(text_embeddings_zh)}개 중국어 쿼리 임베딩 완료!")


=== 중국어 쿼리 임베딩 ===

✓ Query 0: 古典音乐，平静的心情，慢节奏，钢琴和弦乐，无人声，适合阅读或...
  - shape: torch.Size([1, 512])

✓ Query 1: 电子舞曲，充满活力，快节奏，合成器，无人声，适合运动或派对...
  - shape: torch.Size([1, 512])

총 2개 중국어 쿼리 임베딩 완료!


In [9]:
import numpy as np

# L2 norm 검증
sample_emb = text_embeddings_en["query_en_0"].cpu().numpy().squeeze()
l2_norm = np.linalg.norm(sample_emb)

print(f"샘플 쿼리 임베딩 L2 norm: {l2_norm:.6f}")
print(f"임베딩 shape: {sample_emb.shape}")
print(f"임베딩 dtype: {sample_emb.dtype}")

# L2 norm이 1이 아니면 정규화 필요 여부 확인
if not np.isclose(l2_norm, 1.0, atol=1e-3):
    print(f"\n⚠️ L2 norm이 1이 아닙니다. Supabase 검색 전 정규화가 필요할 수 있습니다.")
else:
    print(f"\n✅ L2 norm이 1입니다. 정규화 필요 없음.")


샘플 쿼리 임베딩 L2 norm: 1.000000
임베딩 shape: (512,)
임베딩 dtype: float32

✅ L2 norm이 1입니다. 정규화 필요 없음.


In [10]:
def query_to_embedding_list(query: str, normalize: bool = True) -> list:
    """
    텍스트 쿼리를 Supabase 검색에 사용할 수 있는 list 형태로 변환
    
    Args:
        query: 음악을 설명하는 텍스트 쿼리
        normalize: L2 정규화 수행 여부
        
    Returns:
        임베딩 벡터 (list)
    """
    emb = get_text_embedding(query)
    emb_np = emb.cpu().numpy().squeeze()
    
    if normalize:
        l2_norm = np.linalg.norm(emb_np)
        if l2_norm > 0:
            emb_np = emb_np / l2_norm
    
    return emb_np.tolist()


# 테스트
test_query = "Electronic, chill mood, medium tempo, synth pads, no vocals, suitable for background music"
embedding_list = query_to_embedding_list(test_query)

print(f"쿼리: {test_query}")
print(f"임베딩 타입: {type(embedding_list)}")
print(f"임베딩 길이: {len(embedding_list)}")
print(f"임베딩 샘플 (앞 5개): {embedding_list[:5]}")
print(f"정규화 후 L2 norm: {np.linalg.norm(embedding_list):.6f}")


쿼리: Electronic, chill mood, medium tempo, synth pads, no vocals, suitable for background music
임베딩 타입: <class 'list'>
임베딩 길이: 512
임베딩 샘플 (앞 5개): [0.0007438912289217114, -0.03961394727230072, 0.01705898530781269, 0.03875938802957535, -0.04740126430988312]
정규화 후 L2 norm: 1.000000


In [11]:
def build_music_query(
    genre: str,
    mood: str,
    tempo_or_energy: str,
    main_instruments: str,
    vocal_info: str = "no vocals",
    use_case: str = "background music"
) -> str:
    """
    템플릿에 맞춰 음악 검색 쿼리를 생성하는 헬퍼 함수
    
    Template:
        {genre}, {mood} mood, {tempo or energy}, 
        {main instruments}, {vocal info + language}, 
        suitable for {use-case or scene}
    """
    return f"{genre}, {mood} mood, {tempo_or_energy}, {main_instruments}, {vocal_info}, suitable for {use_case}"


# 헬퍼 함수 사용 예시
query = build_music_query(
    genre="Ambient",
    mood="peaceful",
    tempo_or_energy="slow tempo",
    main_instruments="piano and strings",
    vocal_info="no vocals",
    use_case="meditation or sleep"
)
print(f"생성된 쿼리:\n{query}")
print(f"\n임베딩 shape: {get_text_embedding(query).shape}")


생성된 쿼리:
Ambient, peaceful mood, slow tempo, piano and strings, no vocals, suitable for meditation or sleep

임베딩 shape: torch.Size([1, 512])


## Supabase 검색 테스트

`search_tracks_muq` RPC 함수를 사용하여 MuQ-MuLan 임베딩 기반 음악 검색


In [12]:
import dotenv, os
from supabase import create_client

dotenv.load_dotenv("../../.env.local", override=True)

supabase_client = create_client(
    supabase_url=os.getenv("SUPABASE_URL"),
    supabase_key=os.getenv("SUPABASE_SECRET_KEY"),
)

print("Supabase 클라이언트 연결 완료!")


Supabase 클라이언트 연결 완료!


In [13]:
# 검색할 쿼리 설정
search_query = "the sound of happy pop"
print(f"검색 쿼리: {search_query}\n")

# 쿼리 임베딩 생성
query_embedding = query_to_embedding_list(search_query)
print(f"임베딩 길이: {len(query_embedding)}")


검색 쿼리: the sound of happy pop

임베딩 길이: 512


In [20]:
# Supabase 벡터 검색: search_tracks_muq RPC 함수 사용
# embeddingMuq 필드를 사용한 유사도 검색

response = supabase_client.rpc("search_tracks_muq", {
    "query_embedding": query_embedding,
    "match_count": 5
}).execute()

results = response.data if hasattr(response, 'data') else response

import IPython.display as ipd

print(f"🔍 검색 쿼리: {search_query}")
print(f"📊 검색 결과: {len(results)}개\n")
print("=" * 60)

for i, item in enumerate(results):
    print(f"\n[{i+1}] {item.get('title', 'Untitled')}")
    print(f"태그: {item.get('tags', 'N/A')[:100]}...")
    print(f"유사도 (거리): {item.get('similarity', 'N/A')}")
    
    # audio_url = item.get('audioUrl')
    # if audio_url:
    #     display(ipd.Audio(audio_url, autoplay=False))
    
    print("-" * 60)


🔍 검색 쿼리: the sound of happy pop
📊 검색 결과: 5개


[1] 난 이대로 갈 건데
태그: Energetic Japanese alt-pop/rock with quirky “anime MV” vibe; 127–133 BPM, 4/4, bright punchy mix, Sy...
유사도 (거리): 0.690078148541504
------------------------------------------------------------

[2] seeya
태그: Style: Minimal House with K-pop gloss, 112–118 BPM, bouncy 4-on-the-floor, round sub-bass, muted M1-...
유사도 (거리): 0.701137942143556
------------------------------------------------------------

[3] 잔향
태그: Indie rock / dance-punk, early-2010s UK indie dance groove; bright jangly palm-muted electric guitar...
유사도 (거리): 0.704877767402926
------------------------------------------------------------

[4] 도시의 퍼즐
태그: layered beats with smooth brass sections and rhythmic piano, perfect for a k-pop group blending voca...
유사도 (거리): 0.70575903969285
------------------------------------------------------------

[5] 난 이대로 갈 건데 ver3
태그: Energetic Japanese alt-pop/rock with quirky “anime MV” vibe; 127–133 BPM, 4/4, bright punchy mix

## 다양한 쿼리 테스트

영어 및 중국어 쿼리로 검색 테스트


In [18]:
def search_music_muq(query: str, match_count: int = 5, verbose: bool = True):
    """
    MuQ-MuLan 임베딩을 사용하여 음악을 검색하는 헬퍼 함수
    
    Args:
        query: 검색 쿼리 (영어 또는 중국어)
        match_count: 반환할 결과 수
        verbose: 상세 출력 여부
    
    Returns:
        검색 결과 리스트
    """
    # 쿼리 임베딩 생성
    query_embedding = query_to_embedding_list(query)
    
    # Supabase 검색
    response = supabase_client.rpc("search_tracks_muq", {
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()
    
    results = response.data if hasattr(response, 'data') else response
    
    if verbose:
        print(f"🔍 쿼리: {query}")
        print(f"📊 결과: {len(results)}개\n")
        
        for i, item in enumerate(results):
            print(f"[{i+1}] {item.get('title', 'Untitled')} (거리: {item.get('similarity', 'N/A'):.6f}, )")
        print()
    
    return results


# 영어 쿼리 테스트
print("=== 영어 쿼리 테스트 ===\n")
search_music_muq("relaxing piano music for studying")
search_music_muq("energetic dance music with strong beats")
print()


=== 영어 쿼리 테스트 ===

🔍 쿼리: relaxing piano music for studying
📊 결과: 5개

[1] Poster 2 1 (거리: 0.837191, )
[2] 걸음 ver2 (거리: 0.851389, )
[3] City Serenade (거리: 0.867976, )
[4] Midnight Mosaic (거리: 0.893913, )
[5] little chaos (거리: 0.894991, )

🔍 쿼리: energetic dance music with strong beats
📊 결과: 5개

[1] Poster 2 UK House ver (거리: 0.778109, )
[2] Poster 2 5 (거리: 0.822262, )
[3] 난 이대로 갈 건데 (거리: 0.822917, )
[4] 난 이대로 갈 건데 ver2 (거리: 0.825969, )
[5] loochika (거리: 0.830288, )




In [19]:
# 중국어 쿼리 테스트 (MuQ-MuLan의 강점!)
print("=== 중국어 쿼리 테스트 ===\n")
search_music_muq("轻松的钢琴音乐，适合学习")  # 편안한 피아노 음악, 공부에 적합
search_music_muq("充满活力的舞曲")  # 활력 넘치는 댄스 음악
print()


=== 중국어 쿼리 테스트 ===

🔍 쿼리: 轻松的钢琴音乐，适合学习
📊 결과: 5개

[1] groovy sky (거리: 0.768425, )
[2] Moonlight Bounce (거리: 0.815878, )
[3] Midnight Mosaic (거리: 0.816568, )
[4] 걸음 ver2 (거리: 0.822081, )
[5] City Serenade (거리: 0.828036, )

🔍 쿼리: 充满活力的舞曲
📊 결과: 5개

[1] 난 이대로 갈 건데 ver2 (거리: 0.585205, )
[2] Poster 2 UK House ver (거리: 0.608598, )
[3] loochika (거리: 0.609652, )
[4] Poster 2 5 (거리: 0.634734, )
[5] Poster 2 1 (거리: 0.636279, )


